<a href="https://colab.research.google.com/github/Somsahu20/Langchain/blob/main/Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
!pip install DDGS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 15.2 MB/s eta 0:00:00


In [6]:
from langchain_community.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()
res = search_tool.run("What is langchain?")

print(res)

January 12, 2026 - LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications . November 17, 2025 - LangChain is an open source orchestration framework for the development of applications using large language models (LLMs), like chatbots and virtual agents . 1 week ago - LangChain is an open source framework for building applications based on large language models (LLMs). LLMs are large deep-learning models pre-trained on large amounts of data that can generate responses to user queries—for example, answering questions or creating images from text-based prompts. LangChain is an open-source framework designed to simplify the creation of applications using large language models (LLMs). It provides a standard interface for integrating with other tools and end-to-end chains for common applications. Published December 12, 2025 August 19, 2025 - At its core, LangChain is a framework that handles all the tedious, repetitive stuff 

**Custom Tools**

In [8]:
from langchain_core.tools import tool

In [11]:
@tool
def multiply(a: int, b: int) -> int:
  """ Multiply 2 numbers """ #! Highly recommended to do this
  return a * b

In [12]:
result = multiply.invoke({"a": 10, "b": 20})
print(result)

200


In [13]:
print(multiply.description)
print(multiply.args)

Multiply 2 numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [15]:
#! What LLM sees
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply 2 numbers ', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


# **Using Structured Tools**

In [16]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field

In [17]:
class MultiplyInput(BaseModel):
  a: int = Field(description="First number")
  b: int = Field(description="Second number")

In [18]:
def multiply_func(a: int, b: int) -> int:
  return a * b

In [19]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply 2 numbers",
    args_schema=MultiplyInput
)

In [20]:
result = multiply_tool.invoke({"a": 10, "b": 20})
print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args_schema.model_json_schema())

200
multiply
Multiply 2 numbers
{'properties': {'a': {'description': 'First number', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'Second number', 'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'MultiplyInput', 'type': 'object'}


# **Using BaseTool**

In [22]:
from langchain_core.tools import BaseTool
from typing import Type

In [23]:
class MultiplyTool(BaseTool):
  name: str = "multiply"
  description: str = "Multiply 2 numbers"
  args_schema: Type[BaseModel] = MultiplyInput

  def _run(self, a: int, b: int) -> int:
    return a * b


In [25]:
multiply_tool2 = MultiplyTool()
result = multiply_tool2.invoke({"a": 10, "b": 20})
print(result)
print(multiply_tool2.name)
print(multiply_tool2.description)
print(multiply_tool2.args_schema.model_json_schema())


200
multiply
Multiply 2 numbers
{'properties': {'a': {'description': 'First number', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'Second number', 'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'MultiplyInput', 'type': 'object'}


# **Toolkits**

In [26]:
@tool
def add(a: int, b: int) -> int:
  """ Add 2 numbers """
  return a + b



In [27]:
class MathTool:
  def get_tools(self):
    return [add, multiply]



In [28]:
toolkit = MathTool()
tools = toolkit.get_tools()
print(tools)

[StructuredTool(name='add', description='Add 2 numbers', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x7ca25480f060>), StructuredTool(name='multiply', description='Multiply 2 numbers', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x7ca25480c360>)]
